In [2]:
########################## dataload part###############
from datetime import datetime
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import torch

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

import os
# disable CPU warning
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# lagged GHI values
LAG = 1

# prediction horizon
K = 24

seq_len =24

EXOGENOUS = True

# features
if(EXOGENOUS):
    features = ['K','uvIndex','cloudCover','sunshineDuration','windBearing','humidity','temperature','hour','dewPoint']
else:
    features = ['K']

df = pd.read_csv("../clean_dataset.csv",header=0, index_col=0, parse_dates=True).sort_index()
df_GHI = df[['K']].copy()
# create exogenous regressors
for feature in features:
    df_GHI[feature] = df[feature]
    for i in range(LAG-1):
        df_GHI[feature+'-'+str(i+1)] = df[feature].shift(i+1)
# create target values
for i in range(1,K+1):
    #  df_GHI['K+'+str(i)] = df['K'].shift(-i)
    df_k= df['K'].shift(-i).rename('K+'+str(i))
    df_GHI = pd.concat([df_GHI,df_k],axis=1)

# create clear sky target values
for i in range(1,K+1):
    # df_GHI['GHI_cs+'+str(i)] = df['GHI_cs'].shift(-i)
    df_cs = df['GHI_cs'].shift(-i).rename('GHI_cs+'+str(i))
    df_GHI = pd.concat([df_GHI,df_cs],axis=1)

# drop nan due to shifting
df_GHI = df_GHI.dropna()
    # create training set
X_train = df_GHI['2010-1-1':'2014-6-30'].values[:,:-K*2]
y_train = df_GHI['2010-1-1':'2014-6-30'].values[:,-K*2:-K]

# create validation set
X_val = df_GHI['2014-7-1':'2014-12-31'].values[:,:-K*2]
y_val = df_GHI['2014-7-1':'2014-12-31'].values[:,-K*2:-K]
# create test set
X_test = df_GHI['2015-1-1':'2015-12-31'].values[:,:-K*2]
y_test = df_GHI['2015-1-1':'2015-12-31'].values[:,-K*2:-K]
# get clear sky target values
y_cs = df_GHI['2015-1-1':'2015-12-31'].values[:,-K:]
# scale features
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
X=[]
Y=[]
for i in range(0, X_train.shape[0]-seq_len, seq_len):
  # print('i:',i)
  data_X = np.expand_dims(X_train[i:i + seq_len], 0)
  data_Y = np.expand_dims(y_train[i:i + seq_len,0], 0)
  # print(data_X.shape)  # (1, 24, 9)
  # print(data_Y.shape)  # (1, 24, 24)
  X.append(data_X)
  Y.append(data_Y)
data_X= np.concatenate(X, axis=0)  ##axis=0 按照行拼接，axis=1 按照列拼接
data_Y=np.concatenate(Y, axis=0)
X=[]
Y=[]
for i in range(0, X_val.shape[0]-seq_len, seq_len):
  datav_X = np.expand_dims(X_val[i:i + seq_len], 0)
  datav_Y = np.expand_dims(y_val[i:i + seq_len,0], 0)
  X.append(datav_X)
  Y.append(datav_Y)
X_val= np.concatenate(X, axis=0)  ##axis=0 按照行拼接，axis=1 按照列拼接
y_val=np.concatenate(Y, axis=0)

def data1_load(batch):
    data_train = torch.from_numpy(data_X).type(torch.float32)
    label_train = torch.from_numpy(data_Y).type(torch.float32)
    data_val = torch.from_numpy(X_val).type(torch.float32)
    label_val = torch.from_numpy(y_val).type(torch.float32)
    # data_test = torch.from_numpy(X_test).type(torch.float32)
    # label_test = torch.from_numpy(y_test).type(torch.float32)

    dataset_train = TensorDataset(data_train,label_train)
    datatrain_loader = DataLoader(dataset_train,batch_size=batch,shuffle=False)  # 数据迭代器DataLoader
    dataset_val = TensorDataset(data_val,label_val)
    dataval_loader = DataLoader(dataset_val,batch_size=batch,shuffle=False)
    # dataset_test = TensorDataset(data_test,label_test)


    return datatrain_loader,dataval_loader,LAG
    # return datatrain_loader

X=[]
Y=[]
Y_cs = []
for i in range(0, X_test.shape[0]-seq_len, seq_len):
  # print('i:',i)
  datat_X = np.expand_dims(X_test[i:i + seq_len], 0)
  datat_Y = np.expand_dims(y_test[i:i + seq_len,0], 0)
  datacs_Y = np.expand_dims(y_cs[i:i + seq_len,0], 0)
  X.append(datat_X)
  Y.append(datat_Y)
  Y_cs.append(datacs_Y)
X_test= np.concatenate(X, axis=0)  ##axis=0 按照行拼接，axis=1 按照列拼接
y_test=np.concatenate(Y, axis=0)
y_cs= np.concatenate(Y_cs, axis=0)

def data2_load(batchtest):
    data_test = torch.from_numpy(X_test).type(torch.float32)
    label_test = torch.from_numpy(y_test).type(torch.float32)


    dataset_test = TensorDataset(data_test,label_test)
    datatest_loader = DataLoader(dataset_test,batch_size=batchtest,shuffle=False)
    return datatest_loader,label_test,y_cs

In [3]:
###########model part####################################
import torch,math
import torch.nn as nn
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=64):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)    #64*512
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)    #64*1
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))    #256   model/2
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)# pe.requires_grad = False
        self.register_buffer('pe', pe)   #64*1*512
    def forward(self, x):     #[seq,batch,d_model]
        return x + self.pe[:x.size(0), :]   #64*64*512
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
          ########################自己修改过的########################
            # attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
            attn_scores = attn_scores.masked_fill(torch.tensor(mask == 0), -1e9)
          #############################################################
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, dim_feedforward):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, dim_feedforward)
        self.fc2 = nn.Linear(dim_feedforward, d_model)
        self.relu = nn.ReLU()
    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads,dim_feedforward=2048, dropout=0):
        super(EncoderLayer, self).__init__()
        self.d_model = d_model
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, dim_feedforward)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        self.w0 = torch.nn.Parameter(torch.FloatTensor(1), requires_grad=True)
        self.w1 = torch.nn.Parameter(torch.FloatTensor(1), requires_grad=True)
        self.w2 = torch.nn.Parameter(torch.FloatTensor(1), requires_grad=True)
        self.linear0 = nn.Linear(d_model, d_model)
        self.linear1 = nn.Linear(d_model, d_model)
        self.linear2 = nn.Linear(d_model, d_model)
        self.linear3 = nn.Linear(12288, d_model)  # x.size()[1]*x.size()[2]
        self.linear4 = nn.Linear(9216, d_model)  # attn_output.size()[1]*attn_output.size()[2]
    def forward(self, x, mask=None):
        n_x1 = x.shape[1]
        n_x2 = x.shape[2]
#############################################

        i = 3
        nn_x = n_x1 - i * (i + 1) // 2
        if nn_x <= 0 or nn_x > n_x1:
            attn_output = self.self_attn(x, x, x, mask)
            x = self.norm1(x + self.dropout(attn_output))
            ff_output = self.feed_forward(x)
            x = self.norm2(x + self.dropout(ff_output))
            return x
        else:
            x_part0 = x[:,:nn_x,:]
            # print('x_part0',x_part0.shape)

            attn_output0 = self.self_attn(x_part0, x_part0, x_part0, mask)  # torch.Size([4, 10, 512])
            x_part1 = x[:,3:nn_x+3,:]
            attn_output1 = self.self_attn(x_part1, x_part1, x_part1, mask)
            x_part2 = x[:,6:,:]
            # print('权重：',self.w0,self.w1,self.w2)
            # print('x_part2:',x_part2.shape)

            attn_output2 = self.self_attn(x_part2, x_part2, x_part2, mask)
            # print('attn_output012的结果已经计算好了')
            attn_output = torch.mul(self.w0, self.linear0(attn_output0)) + \
                      torch.mul(self.w1, self.linear1(attn_output1)) + \
                      torch.mul(self.w2, self.linear2(attn_output2))
            # print('attn_output的结果已经计算好了')
            x = self.linear3(x.view(x.size()[0],x.size()[1]*x.size()[2]))
            attn_output = self.linear4(attn_output.view(attn_output.size()[0],attn_output.size()[1]*attn_output.size()[2]))
            x = self.norm1(x + attn_output)
            ff_output = self.feed_forward(x)
            x = self.norm2(x + self.dropout(ff_output))
            return x

        # x_part0 = x[:,:10,:]
        # attn_output0 = self.self_attn(x_part0, x_part0, x_part0, mask)  # torch.Size([4, 10, 512])
        # x_part1 = x[:,3:13,:]
        # attn_output1 = self.self_attn(x_part1, x_part1, x_part1, mask)  # torch.Size([4, 10, 512])
        # x_part2 = x[:,6:,:]
        # attn_output2 = self.self_attn(x_part2, x_part2, x_part2, mask)  # torch.Size([4, 10, 512])
        # attn_output = torch.mul(self.w0, self.linear0(attn_output0)) + \
        #               torch.mul(self.w1, self.linear1(attn_output1)) + \
        #               torch.mul(self.w2, self.linear2(attn_output2))    # torch.Size([4, 10, 512])
        # x = self.linear3(x.view(x.size()[0],x.size()[1]*x.size()[2]))
        # # print(self.w0,self.w1,self.w2)
        # attn_output = self.linear4(attn_output.view(attn_output.size()[0],attn_output.size()[1]*attn_output.size()[2]))
        # x = self.norm1(x + attn_output)
        # ff_output = self.feed_forward(x)
        # x = self.norm2(x + self.dropout(ff_output))
        # return x

class TransformerRegressor(nn.Module):
    def __init__(self, input_dim, output_dim, num_heads, d_model):
        super(TransformerRegressor, self).__init__()
        self.embedding = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        ####这里就要开始考虑x.shape
        self.transformer = EncoderLayer(d_model,num_heads)
        self.linear = nn.Linear(d_model, output_dim)

    def forward(self, x):
        # print(x.shape)  # torch.Size([32, 1, 9])
        x = self.embedding(x)  # torch.Size([32, 1, 512])
        # print('输入的shape:',x.shape)
        x = self.pos_encoder(x)  # torch.Size([4, 16, 512])
        x = self.transformer(x)  # torch.Size([4, 512])
        x = self.linear(x)
        return x


In [4]:
###################train part##########################
# from model import TransformerRegressor
import torch
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
import torch.optim as optim
from matplotlib import pyplot
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from datetime import datetime
import time
import os
# from data import datatrain_load,dataval_load

# from data2 import data1_load,data2_load
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
writer = SummaryWriter('./logs/')

# prediction horizon
K = 24


#epochs
epochs = 500


# use exogenous inputs
EXOGENOUS = True

# features
if(EXOGENOUS):
    features = ['K','uvIndex','cloudCover','sunshineDuration','windBearing','humidity','temperature','hour','dewPoint']
else:
    features = ['K']


# metrics
def mad(y_pred,y_test):
    return 100 / y_test.mean() * np.absolute(y_pred - y_test).sum() / y_pred.size

def mdb(y_pred,y_test):
    return 100 / y_test.mean() * (y_pred - y_test).sum() / y_pred.size

def r2(y_pred,y_test):
    return r2_score(y_test, y_pred)

def rmsd(y_pred,y_test):
    return 100 / y_test.mean() * np.sqrt(np.sum(np.power(y_pred - y_test, 2)) / y_pred.size)

def mae(y_pred,y_test):
    return mean_absolute_error(y_test, y_pred)

def mse(y_pred,y_test):
    return mean_squared_error(y_test, y_pred)

In [5]:
train_loader,val_loader,lag= data1_load(32)
test_loader,label_test,y_cs= data2_load(1)
num_clo = y_cs.shape[0]
Num_f = len(features)
input_d= Num_f*lag
model = TransformerRegressor(input_dim=input_d,output_dim=K,num_heads=8,d_model=512).to(device)
criterion = nn.MSELoss().to(device)     # 忽略 占位符 索引为0.
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.99)
val_loss = []
train_loss = []
best_loss = 10000
y_preds = np.zeros(y_cs.shape)

In [18]:
def train_val():
    # j = 0
    best_test_loss = 100000
    for epoch in range(epochs):
        # if j >=100:
        #     break
        # else: j +=1
        train_epoch_loss = []
        val_epoch_loss = []
        for X_train, Y_train in train_loader:
            y= Y_train
            # y = Y_train[:,i]
            #####################################################################################
            # y = y.unsqueeze(1)
            # X_tr = np.expand_dims(X_train,0)
            # y_tr = np.expand_dims(y,0)
            # input_xtr = X_tr.transpose(1, 0, 2) #数组转维度，(1,32,9) -->(32,1,9)
            #########################################################################################
            input_xtr=X_train.clone().detach().requires_grad_(True)
            # print('input_xtr:',input_xtr.shape)
            input_ytr = Y_train.clone().detach().requires_grad_(True)
            input_xtr = input_xtr.clone().detach().to(device)
            input_ytr = input_ytr.clone().detach().to(device)
            optimizer.zero_grad()
            output = model(input_xtr).to(device)#训练
            trainloss = criterion(output, input_ytr).to(device)
            trainloss.backward()
            optimizer.step()
        train_epoch_loss.append(trainloss.item())
        train_loss.append(np.mean(train_epoch_loss))
        writer.add_scalar("train_loss", np.mean(train_epoch_loss))
        for X_val, y_val in val_loader:
          ############################################################
            # y_val = y_val
            # y_val = y_val.unsqueeze(1)
            # x_v = np.expand_dims(X_val, 0)
            # y_v = np.expand_dims(y_val, 0)
            # input_xval = x_v.transpose(1, 0, 2)
            # input_yval = y_v.transpose(1, 0, 2)
            # X_val = torch.tensor(input_xval).to(device)
            # Y_val = torch.tensor(input_yval).to(device)
          ################################################################
            X_val = X_val.clone().detach().to(device)
            Y_val = y_val.clone().detach().to(device)
            output = model(X_val).to(device)
            valloss = criterion(output, Y_val)
            val_epoch_loss.append(valloss.item())
        val_loss.append(np.mean(val_epoch_loss))
        writer.add_scalar("val_loss", np.mean(val_epoch_loss), epoch)
        datetimeStr = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(datetimeStr +"  Epoch【" + str(epoch) +  "】 " +  "train_loss:" + str(round(np.mean(train_epoch_loss), 5)) + "  val_loss:" + str(round(np.mean(val_epoch_loss), 5)))

        if np.mean(val_epoch_loss) < best_test_loss:
            # j = 0
            best_test_loss = np.mean(val_epoch_loss)
            best_model = model
            print("best_test_loss:", best_test_loss)
            torch.save(best_model.state_dict(), './logs/best_Transformer_trainModel.pth')

In [7]:
train_val()


2023-10-25 15:47:10  Epoch【0】 train_loss:0.15393  val_loss:0.13674
best_test_loss: 0.13673514322094296
2023-10-25 15:47:12  Epoch【1】 train_loss:0.08811  val_loss:0.0929
best_test_loss: 0.09289665406812793
2023-10-25 15:47:14  Epoch【2】 train_loss:0.06286  val_loss:0.062
best_test_loss: 0.06200489029288292
2023-10-25 15:47:16  Epoch【3】 train_loss:0.04152  val_loss:0.09861
2023-10-25 15:47:18  Epoch【4】 train_loss:0.03368  val_loss:0.11603
2023-10-25 15:47:20  Epoch【5】 train_loss:0.19656  val_loss:0.73568
2023-10-25 15:47:22  Epoch【6】 train_loss:1.51347  val_loss:4.12466
2023-10-25 15:47:23  Epoch【7】 train_loss:0.19603  val_loss:0.14757
2023-10-25 15:47:25  Epoch【8】 train_loss:0.06234  val_loss:0.21481
2023-10-25 15:47:27  Epoch【9】 train_loss:0.15549  val_loss:0.12738
2023-10-25 15:47:30  Epoch【10】 train_loss:0.0845  val_loss:0.08103
2023-10-25 15:47:32  Epoch【11】 train_loss:0.04991  val_loss:0.17251
2023-10-25 15:47:34  Epoch【12】 train_loss:0.07517  val_loss:0.13237
2023-10-25 15:47:36  E

In [13]:
####test部分
def test():
    model.load_state_dict(torch.load('./logs/best_Transformer_trainModel.pth'))
    model.to(device)
    model.eval()
    y_pred = []
    for X_test,y_test in test_loader:
        X_tt = X_test.clone().detach().requires_grad_(True)
        # Y_tt = y_test.clone().detach().requires_grad_(True)
        output = model(X_tt.to(device)).to(device)
        # output = output.cpu().detach().numpy()[0]
        y_pred.append(output)
        # Y_tt = Y_tt.cpu().detach().numpy()[0] #将Y_tt从GPU移动到CPU，并释放GPU上的内存,并保存第一个值（这里本来是为了做对比留下来的，但是后面直接取值label_test，最后没用到）
    # print('test预测')
    pred = [tensor.detach().cpu().numpy() for tensor in y_pred]
    pred = np.array(pred)
    y_preds = pred
    return y_preds

In [14]:
print('lag:',lag)

lag: 1


In [15]:
preds = test()
preds = preds.reshape(-1, 24)

In [16]:
print('preds:',preds)
print('preds.shape:',preds.shape)
print('y_cs:',y_cs.shape)

preds: [[0.9970426  0.9999771  0.99946845 ... 0.99534535 1.0155811  1.0129863 ]
 [1.0029305  1.0096799  0.99860317 ... 0.9786899  1.0222156  1.0397909 ]
 [1.001255   0.9993527  1.0016567  ... 0.992277   1.0138973  1.0500742 ]
 ...
 [1.0006012  0.99981546 1.000529   ... 0.98730683 1.0162315  1.0347909 ]
 [0.99995047 1.0010455  0.99757874 ... 0.9964837  1.0223261  0.9711082 ]
 [1.0026038  0.9986069  0.99717957 ... 0.458621   0.52386266 0.5158485 ]]
preds.shape: (1458, 24)
y_cs: (1458, 24)


In [17]:
# transform to GHI

y_pred2 = np.multiply(preds, y_cs)

y_test2 = np.multiply(label_test,y_cs)
y_test2 =y_test2.numpy()
# save results
results = {'K':[],'Time[min]':[],'MAD[%]':[],'R2':[],'RMSD[%]':[]}
for i in range(K):
    results['K'].append(i+1)
    results['Time[min]'].append((i+1)*15)
    results['MAD[%]'].append(np.round(mad(y_pred2[:,i],y_test2[:,i]),2))
    results['R2'].append(np.round(r2(y_pred2[:,i],y_test2[:,i]),5))
    results['RMSD[%]'].append(np.round(rmsd(y_pred2[:,i],y_test2[:,i]),2))

# create results dataframe
results = pd.DataFrame(results)
results = results.set_index('K')
print(results.head(K))

    Time[min]  MAD[%]       R2  RMSD[%]
K                                      
1          15    1.56  0.99931     4.43
2          30    1.84  0.99885     5.63
3          45    1.66  0.99927     4.37
4          60    1.38  0.99950     3.51
5          75    2.56  0.99748     7.61
6          90    4.34  0.99216    13.01
7         105    3.74  0.99363    11.49
8         120    4.32  0.99153    13.09
9         135    3.73  0.99443    10.51
10        150    3.69  0.99476    10.20
11        165    3.21  0.99609     8.86
12        180    3.64  0.99476    10.47
13        195    3.13  0.99640     8.92
14        210    3.19  0.99617     9.41
15        225    1.92  0.99879     5.46
16        240    1.06  0.99976     2.51
17        255    1.41  0.99950     3.70
18        270    1.57  0.99927     4.55
19        285    1.75  0.99902     5.36
20        300    1.85  0.99895     5.61
21        315    2.30  0.99843     6.86
22        330    2.26  0.99866     6.36
23        345    1.47  0.99961     3.42
